# Protein Bioinformatics - Day 1



In [2]:
import sys
import os
import re

from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import Entrez
from Bio.Align import MultipleSeqAlignment as MSA
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

# To configure
Entrez.email = "gda.gaming.off@gmail.com"
assert Entrez.email != ""

# Hidde warnings
import warnings
warnings.filterwarnings('ignore')

## Exercise 1
In a DNA sequence, identify the possile genes and it's transcripted and translated products. Identify at least the most similar protein with a hit in the protein data bank for each translated protein.

In [3]:
def possible_cds_pos(seq, start_codon=["ATG"], stop_codon=["TAA", "TAG", "TGA"]):
    """
    return the pair (start, stop) for possible cds with stop - start > 6
    """
    start_pos = []
    stop_pos = []
    for codon in start_codon:
        start_pos += [m.start() for m in re.finditer(codon, str(seq))]
    for codon in stop_codon:
        stop_pos += [m.start()+3 for m in re.finditer(codon, str(seq))]
        
    cds_pos = [(start, stop) for start in start_pos for stop in stop_pos if stop - start > 6]
    return cds_pos

def possible_cds_seq(seq, start_codon=["ATG"], stop_codon=["TAA", "TAG", "TGA"]):
    """
    extract the sequence of possible cds in a sequence
    """
    cds_pos = possible_cds_pos(seq, start_codon, stop_codon)
    cds_seq = [Seq(str(seq[pos[0]:pos[1]])) for pos in cds_pos]
    
    return cds_seq

In [40]:
cds_seq = possible_cds_seq(Seq("ATGGTTTGTAGAGCTATACGGGCCAGAGATC")) #CDS array
cds_rna = [cds.transcribe() for cds in cds_seq] #CDS as RNA array
cds_prot = [cds.translate() for cds in cds_seq] # CDS as prot array

# Loop and query blastp
for prot in cds_prot:
    query = NCBIWWW.qblast("blastp", "refseq_protein", prot)
    result = NCBIXML.read(query)
    
    for alignment in result.alignments:
        for hsp in alignment.hsps:
            if hsp.expect < 0.0001:
                print('*********alignment**********')
                print('Sequence: ', alignment.title)
                print('E value: ', hsp.expect)
                print("Bit score: ", hsp.bits)

## Exercise 2
From a list of genebank identifiers, retreive their corresponding protein sequences in fasta format. After, indicate if the proteins have been experimentally validated and retrieve the related literature.

In [4]:
# Main function
def retrieve_from_genebank(gb_ids, format=None):
    if isinstance(gb_ids, str):
        return retrieve_from_genebank([gb_ids], format)
    elif isinstance(gb_ids, (list, tuple)):
        return [_retrieve_from_genebank_(gb_id, format) for gb_id in gb_ids]
    else:
        raise ValueError()

# Function to handle single id
def _retrieve_from_genebank_(gb_id, format=None):
    # Load data from NCBI using Bio.Entrez & read data as GenBank data
    handle = Entrez.efetch(db="nucleotide", id=gb_id, rettype="gb", retmode="text")
    gb_data = SeqIO.read(handle, 'genbank')

    # Convert DNA Seq to Protein Seq and format as fasta
    dna_seq = gb_data.seq
    prot_seq = gb_data.seq.translate()
    prot_rec = SeqRecord(prot_seq, id=gb_data.id, name=gb_data.name, description=gb_data.description, dbxrefs=gb_data.dbxrefs)
    
    if format:
        prot_rec = prot_rec.format(format)
    
    # Verifiy if data was validated, if so return the list of references
    if gb_data.annotations['references']:
        return (prot_rec, True, gb_data.annotations['references'])
    return (prot_rec, False, [])

In [5]:
retrieve_from_genebank(('X14061', 'X15062'), format="fasta")

[('>X14061.1 M.musculus beta-globin complex DNA for y, bh0, bh1, b1 and b2 genes, bh2 and bh3 pseudogenes\nEFIADIHWYPVSLSDLL*LGKLMKSLV*KRKGNNLCC*S*GH*ICTR*FPDKFTFTL*SS\nTEP*FRLESVL*ILVK*ILPFSHHCEHERYSSGM*EYLQKDR*IKGVQMHIWRKKRKHPV\nLLSYSTATASEPKAFSAQMCFL*LFCHLCFLSSKKE*TSSS*KAWLLSTSYHP*KILPDI\nVLTKVLICRNVPKMIFFCDCESFFVLFCFLFFSIFIRYLAHLHFQCYTKSPPYPPTPTPL\nPTHSPFLGLAFPCTGAYKVCKSNGPLFAVMPTRPSFDTYPARDKSSGVLLSSYCCSTYRV\nAVPFSSLGAFSSSSIGGPVVNSIADCEHPLLCLLGPGIVSQETAISGSFQQNLASVCNGV\nSVWKLIMGWISGYGNH*MVHPFVTLPNFVSVTPSMGVLFPILRRGKVSTLWSLFS*V*SF\n*QIVSYILGILSFWANIHLSVSTYCESSFVIGLPHSG*CPPGPSIWLGIS*IHSF**LSS\nTLLCRCTTFSVSIPLLRGIWVLSSFWLL*IRLL*T*WSMCPSYWLGHLLDICPEEVLLDL\nPVVLCPIF*ITTRLISRVVVQACNPTNNGGVFLFLHILASICCHLNF*S*PF*LV*GGIS\nGLF*FAFP**LRMLNNFFRCFSAIRYS*GENSLFSSEPHFLMGLFDFMESTFLNSLYMLD\nISPLSDLG*VKILSQSVGGLFVLLTVSFALQKLCNFMRSHLLILDLTAQAIAVLFRNFSP\nVPISLRLFPTFSSISFSVSGFMWRSLIHLDLTLVQGDSTGSIRNLLHDNSQLCQHHLLKM\nLSFFHWMVLAPLSKIK*PSKDDF*FPI**NFQGIS*VMIIITTGIDRG*HKFTAY*ASKF\nL**YMLQFRN*QALVGA*VCAIMAG

## Exercise 3
Create a theoretical framework for directed mutagenesis of a user specified gene in genbank. The user must be able to decide the mutation position and choose the codon to use. From this codon, create primers with 40% GC content and a minimum of 25bp.

## Exercise 4
Starting with a list of gene identifiers, saved in a file, perform the MSA of the corresponding proteins. From the MSA, retrieve the zones with higher conservation.

In [6]:
handle = 'gb_ids.txt' #specify the location of your gene ids file

def get_ids(handle):
    if not os.path.isfile(handle):
        raise ValueError()

    with open(handle) as f:
        ids = list({line.rstrip() for line in f.readlines()})
            
    return ids

def run_msa(records):
    align = MSA([])
    maxLen = max([len(rec.seq) for rec in records])
    for rec in records:
        # add a padding to the sequences if they dont have the same length
        align.add_sequence(rec.id, str(rec.seq).ljust(maxLen, '-'))
        
    return align

In [7]:
records = [el[0] for el in retrieve_from_genebank(get_ids(handle))]
align = run_msa(records)
print(align.format("phylip"))

 2 18618
X15062.1   SKSCVLIEVH WSANRVARQ* THLD*F*SFV ERLAEN*PEH VWS*SSGKNP
X14061.1   EFIADIHWYP VSLSDLL*LG KLMKSLV*KR KGNNLCC*S* GH*ICTR*FP

           GRQYGTTRSS LLVKQNKTKN KTNWKQTWTF KRCSRIYLFL FVQHFDWKKD
           DKFTFTL*SS TEP*FRLESV L*ILVK*ILP FSHHCEHERY SSGM*EYLQK

           HSPPKEVVEN AGPKTRLGCS KESQESGGQF DERIVLKETP FP*CSDCAIP
           DR*IKGVQMH IWRKKRKHPV LLSYSTATAS EPKAFSAQMC FL*LFCHLCF

           NFGNAVDDGW SDLGAEKQMV APKCDI*GPR ENILCGHRQL HNKHFGSQVL
           LSSKKE*TSS S*KAWLLSTS YHP*KILPDI VLTKVLICRN VPKMIFFCDC

           VPRLNGIQLS QSQSKRGAR* H*LLVLWGGK R*SRIW*V*L SRQV*EVKKG
           ESFFVLFCFL FFSIFIRYLA HLHFQCYTKS PPYPPTPTPL PTHSPFLGLA

           H*LAYA*KPW FEDPARKMDD WKNG*KATPK D*EMVREEPL FCSDGSDHCL
           FPCTGAYKVC KSNGPLFAVM PTRPSFDTYP ARDKSSGVLL SSYCCSTYRV

           PCGKQHDATS RDCPTGLGCW SGLLSSLHWN Y*QGFH*GGA WRNLGFSYPG
           AVPFSSLGAF SSSSIGGPVV NSIADCEHPL LCLLGPGIVS QETAISGSFQ

           ARQVCHCYGP *QAFIGHLTR DSSH**TC*G EESVLQCSSH SCED*

## Exercise 5 (Incomplete)
Starting with a gene sequence, provide the genetic analysis of the sequence (GC content, length, taxonomy and condon usage). Provide also the protein sequence and the corresponding characteristics according to Prosite, if any.

In [8]:
gc_content = lambda seq: (seq.count("G")+seq.count("C"))/len(seq)*100

def seq_stats(seq, source=None, verbose=True):
    stats = [None, None, None, None]
    if source is None:
        stats[0] = gc_content(seq)
        stats[1] = len(seq)
        stats[2] = None #TODO
        stats[3] = None #TODO
        
    # pretty print
    if verbose:
        print(f"> Sequence Stats (Source:{source})\n" + \
              f"\tGC-content.................{stats[0]:.2f}\n" + \
              f"\tLength.....................{stats[1]}\n" + \
              f"\tTaxonomy...................{stats[2]}\n" + \
              f"\tCodon Usage................{stats[3]}\n")

    return stats

seq_stats("ATGCCCGATAGGCTTAAATGAGAGATCGATACAGATAGACCCAATTAAATGAGAGAGATCAGCGCATG", verbose=False)

[42.64705882352941, 68, None, None]

## Exercise 6
Starting from one protein sequence, retrieve the homologous from 5 different species. perform the MSA and retrieve an identity matrix.

## Exercise 7
From a list of proteins, listed by their uniprot identifiers, retreive their EC number and sequence. If they share common enzymatic class on the higher instance, perform the MSA and identify the two most distant proteins.

## Exercise 8
From a list of proteins, listed by their uniprot identifiers, retrieve their PDB code and sequence. Only if they belong to different organisms, perform a MSA and print a phylogenetic tree.